## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.80,87,75,4.61,broken clouds
1,1,San Jeronimo,PE,-11.9472,-75.2829,46.69,81,69,2.35,broken clouds
2,2,Ternate,ID,0.8000,127.4000,83.64,74,81,11.14,broken clouds
3,3,Ormara,PK,25.2088,64.6357,82.51,66,70,10.83,broken clouds
4,4,Novikovo,RU,51.1537,37.8879,47.68,66,93,7.54,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float((input("What is the minimum temperature you would like for your trip? ")))
max_temp = float((input("What is the maximum temperature you would like for your trip? ")))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Ternate,ID,0.8000,127.4000,83.64,74,81,11.14,broken clouds
3,3,Ormara,PK,25.2088,64.6357,82.51,66,70,10.83,broken clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,75.00,99,80,3.29,broken clouds
10,10,Faanui,PF,-16.4833,-151.7500,82.35,73,30,9.33,scattered clouds
11,11,Busselton,AU,-33.6500,115.3333,77.00,44,0,1.99,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                177
City                   177
Country                176
Lat                    177
Lng                    177
Max Temp               177
Humidity               177
Cloudiness             177
Wind Speed             177
Current Description    177
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                176
City                   176
Country                176
Lat                    176
Lng                    176
Max Temp               176
Humidity               176
Cloudiness             176
Wind Speed             176
Current Description    176
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ternate,ID,83.64,broken clouds,0.8000,127.4000,
3,Ormara,PK,82.51,broken clouds,25.2088,64.6357,
9,Puerto Ayora,EC,75.00,broken clouds,-0.7393,-90.3518,
10,Faanui,PF,82.35,scattered clouds,-16.4833,-151.7500,
11,Busselton,AU,77.00,clear sky,-33.6500,115.3333,
17,Arraial Do Cabo,BR,75.34,overcast clouds,-22.9661,-42.0278,
20,Rikitea,PF,77.11,scattered clouds,-23.1203,-134.9692,
21,Waingapu,ID,86.07,overcast clouds,-9.6567,120.2641,
29,Arlit,NE,83.88,overcast clouds,18.7369,7.3853,
31,Hithadhoo,MV,82.47,broken clouds,-0.6000,73.0833,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        pass

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ternate,ID,83.64,broken clouds,0.8000,127.4000,Grand Dafam Bela Ternate
3,Ormara,PK,82.51,broken clouds,25.2088,64.6357,Crystal Beach and Camping Resort Ormara
9,Puerto Ayora,EC,75.00,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Faanui,PF,82.35,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
11,Busselton,AU,77.00,clear sky,-33.6500,115.3333,Observatory Guest House


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ternate,ID,83.64,broken clouds,0.8000,127.4000,Grand Dafam Bela Ternate
3,Ormara,PK,82.51,broken clouds,25.2088,64.6357,Crystal Beach and Camping Resort Ormara
9,Puerto Ayora,EC,75.00,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Faanui,PF,82.35,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
11,Busselton,AU,77.00,clear sky,-33.6500,115.3333,Observatory Guest House
17,Arraial Do Cabo,BR,75.34,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia
20,Rikitea,PF,77.11,scattered clouds,-23.1203,-134.9692,Pension Maro'i
21,Waingapu,ID,86.07,overcast clouds,-9.6567,120.2641,Pondok Wisata Elim
29,Arlit,NE,83.88,overcast clouds,18.7369,7.3853,Case de Passage De la Prefecture
31,Hithadhoo,MV,82.47,broken clouds,-0.6000,73.0833,Scoop Guest House


In [17]:
# 8a. Create the output File (CSV)
output_data_file = 'Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</d>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[['Lat', 'Lng']]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))